In [3]:
%logstop
%logstart -ortq ~/.logs/nlp.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the results of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [5]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [6]:
import gzip
import simplejson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [7]:
ratings = [d["overall"] for d in data]
ratings[:10]

[5.0, 1.0, 4.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0]

In [8]:
data[0]

{'reviewerID': 'A238V1XTSK9NFE',
 'asin': 'B00004VX3T',
 'reviewerName': 'Andrew Lynn',
 'helpful': [2, 2],
 'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatical

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [9]:
from sklearn.base import BaseEstimator,TransformerMixin

class KeySelector (BaseEstimator,TransformerMixin):
    def __init__(self,key):
        self.key = key
    
    def fit(self,x,y = None):
        return self 
    
    def transform(self,x):
        return [d[self.key] for d in x]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
bag_of_words_model = Pipeline([
    ("selector",KeySelector("reviewText")),
    ("vectorizer",HashingVectorizer(n_features = 2_000)),
    ("regressor",LinearRegression())
])

In [11]:
bag_of_words_model.fit(data, ratings)

Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer', HashingVectorizer(n_features=2000)),
                ('regressor', LinearRegression())])

In [12]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score: 1.369


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalization to your model to improve performance.

In [15]:
HashingVectorizer

sklearn.feature_extraction.text.HashingVectorizer

In [16]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

import numpy as np

gs = GridSearchCV(Ridge(),param_grid = {"alpha": np.arange(0.5,2.1,0.5)},
                  cv = 3,
                  n_jobs =1,
                  verbose = 1
                 )
normalized_model = Pipeline([
    ("selector",KeySelector("reviewText")),
    ("vectorizer",HashingVectorizer(norm = "l2",n_features=30_000)),
#    ("regressor",LinearRegression()),
    ("gs",Ridge())
])

In [17]:
normalized_model.fit(data,ratings);

In [18]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score: 1.018


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
bigrams_model = Pipeline([
    ("selector",KeySelector("reviewText")),
    ("vectorizer",HashingVectorizer(lowercase=True,ngram_range=(1,2),
                                   n_features = 30_000)),
#    ("transformer",TfidfTransformer()),
    ("regressor",gs)
])
bigrams_model.fit(data, ratings)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   24.7s finished


Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer',
                 HashingVectorizer(n_features=30000, ngram_range=(1, 2))),
                ('regressor',
                 GridSearchCV(cv=3, estimator=Ridge(), n_jobs=1,
                              param_grid={'alpha': array([0.5, 1. , 1.5, 2. ])},
                              verbose=1))])

In [21]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score: 1.063


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the word "terrible" appears in the text, given that the review is negative. Using these probabilities, we can derive a **polarity score** for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [22]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

File ‘./data/amazon_one_and_five_star_reviews.json.gz’ already there; not retrieving.



In order to avoid memory issues, let's delete the older data.

In [23]:
del data, ratings

In [25]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [d['overall'] for d in data_polarity]

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
pipe = Pipeline([
    ("selector",KeySelector("reviewText")),
    ("vectorizer",TfidfVectorizer( stop_words = "english")),
    ("classifier",MultinomialNB())
])

In [27]:
pipe.fit(data_polarity, ratings)

Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer', TfidfVectorizer(stop_words='english')),
                ('classifier', MultinomialNB())])

In [28]:
log_prob = pipe.named_steps["classifier"].feature_log_prob_
feat_names = pipe.named_steps["vectorizer"].get_feature_names()
polarity = log_prob[0,:] - log_prob[1,:]

In [29]:
terms = sorted(list(zip(polarity,feat_names)))
most_polarized = terms[:25] + terms[-25:]
top_50 = sorted([word for score, word in most_polarized])

In [30]:
grader.score.nlp__most_polar(top_50)

Your score: 1.000


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [31]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*